# microPython gateway

De onderstaande code voor de gateway zou zowel op de ESP32 als op de raspberry pi Pico moeten werken.

```Python
from machine import UART
import time

in_buf = b''       # of: b'' - een bytes object ter lengte 0
LF = b'\n'         # werkt dit?

timer = time.ticks_add(time.ticks_ms(), 5000)

uart = UART(2, baudrate=9600, tx=17, rx=16)
pos = -1

while True:
    if uart.any():
        in_buf = in_buf + uart.read()
        pos = in_buf.find(LF)

    # pos = in_buf.find(LF)
    if pos >= 0 :                    # complete line found
        print(in_buf[0:pos])         # handle first line
        in_buf = in_buf[pos+1:]      # and keep the rest
        pos = in_buf.find(LF)

    if time.ticks_diff(time.ticks_ms(), timer) >= 0:
        uart.write(b'Hello World\nHello\n\nhi...\n')
        timer = time.ticks_add(timer, 5000)
```

De bestanden `boot.py` en `main.py` worden in het ESP filesysteem bewaard (via Thonny).

Hieronder: boot.py

```Python
# This file is executed on every boot (including wake-boot from deepsleep)
#import esp
#esp.osdebug(None)
#import webrepl
#webrepl.start()

# Complete project details at https://RandomNerdTutorials.com

import time
from umqttsimple import MQTTClient
import ubinascii
import machine
import micropython
import network
import esp
esp.osdebug(None)
import gc
gc.collect()

ssid = b'xxx'
password = b'xxx'
mqtt_server = 'infvopedia.nl'
#EXAMPLE IP ADDRESS
#mqtt_server = '192.168.1.144'
client_id = ubinascii.hexlify(machine.unique_id())
topic_sub = b'node/+/actuators'
topic_pub = b'hello'

last_message = 0
message_interval = 5
counter = 0

print('test WiFi - 0')

station = network.WLAN(network.STA_IF)

station.active(True)
station.disconnect()

if not station.isconnected():
    station.connect(ssid, password)

    while station.isconnected() == False:
        time.sleep(1)  

print('Connection successful')
print(station.ifconfig())

```


Hieronder: main.py

(Het is mij nog niet duidelijk wat een handige verdeling is tussen boot.py en main.py. Wat zijn de algemene regels daarvoor, in microPython?)

```Python
# Complete project details at https://RandomNerdTutorials.com

from umqttsimple import MQTTClient, MQTTException
from ulpp import bytes_to_dict, dict_to_bytes
from machine import UART
import json 

uart = UART(2, baudrate=9600, tx=17, rx=16)
pos = -1
in_buf = b''
LF = b'\n'

local_nodes = []

hex = '0123456789ABCDEF'

def nibble(char):
    if char >= ord('0') and char <= ord('9'):
        return char - ord('0')
    elif char >= ord('A') and char <= ord('F'):
        return 10 + char - ord('A')
    elif char >= ord('a') and char <= ord('f'):
        return 10 + char - ord('a')
    else:
        return -1

def hex_to_bytes(data):
    # input bytes or string; only hex chars; even length
    # output bytes
    if type(data) == str:
        data = bytes(data, 'ascii')
    res = []
    assert(len(data) % 2 == 0)
    i = 0
    while i+1 < len(data):
        byte = nibble(data[i]) * 16 + nibble(data[i+1])
        i += 2
        res.append(byte)
    return bytes(res)

def bytes_to_hex(data):
    # input: bytes or int list
    # output: hex bytes
    res = ''
    for i in data:
        res += hex[i // 16] + hex[i % 16]
    return bytes(res, 'ascii')            

def sub_cb(topic, msg, ret, dup):
    print((topic, msg))
    if topic == b'notification' and msg == b'received':
        print('ESP received hello message')
    items = topic.decode('ascii').split('/')
    if items[0] == 'node' and items[2] == 'actuators':
        nodeID_hex = items[1]
        nodeID_bytes = hex_to_bytes(nodeID_hex)
        nodeID = nodeID_bytes[0] * 256 + nodeID_bytes[1]
        if nodeID in local_nodes:
            # convert msg from JSON to dict
            # what is the msg-format? only payload
            lpp_dict = json.loads(msg.decode('ascii'))
            # counter = msg['counter']
            # convert (lpp)dict to (lpp)bytes
            lpp_bytes = dict_to_bytes(lpp_dict)
            # construct msg: tag(1?), nodeID, counter?, lpp-bytes
            counter = 0
            msg = bytes([1, nodeID // 256, nodeID % 256, counter // 256, counter % 256])
            msg = msg + lpp_bytes
            # convert msg to msg_hex
            msg_hex = bytes_to_hex(msg)
            print(msg_hex)
            # send msg_hex to serial
            msg_hex = msg_hex + LF
            uart.write(msg_hex)

def connect_and_subscribe():
    global client_id, mqtt_server, topic_sub
    client = MQTTClient(client_id, mqtt_server, user="mqtttest", password="testmqtt")
    client.set_callback(sub_cb)
    client.connect()
    client.subscribe(topic_sub)
    print('Connected to %s MQTT broker, subscribed to %s topic' % (mqtt_server, topic_sub))
    return client

def restart_and_reconnect():
    print('Failed to connect to MQTT broker. Reconnecting...')
    time.sleep(10)
    machine.reset()

def sensordata_to_mqtt(data):
    port = data[0]    
    if port == 1:
        nodeID = data[1] * 256 + data[2]
        if not nodeID in local_nodes:
            local_nodes.append(nodeID)
            
        nodeID_hex = '{0:x}'.format(nodeID)
        #(x, y) = data[1], data[2]
        # nodeID = hex[x:x+1] + hex[y:y+1]
        counter = data[3] * 256 + data[4]
        lpp_dict = bytes_to_dict(data[5:])
        msg = {'nodeid': nodeID_hex,
               'counter': counter,
               'payload': lpp_dict
              }
        print(json.dumps(msg))
        client.publish(f'node/{nodeID_hex}/sensors', json.dumps(msg))
    else:
        print('???')

try:
    client = connect_and_subscribe()
except MQTTException:
    restart_and_reconnect()    
except OSError as e:
    restart_and_reconnect()

while True:
    try:
        client.check_msg()
        
        if uart.any():
            in_buf = in_buf + uart.read()
            pos = in_buf.find(LF)

        # pos = in_buf.find(LF)
        if pos >= 0 :                    # complete line found
            print(in_buf[0:pos]) # handle first line
            if nibble(in_buf[0]) >= 0 and nibble(in_buf[0]) < 16: 
#            if in_buf[0] != ord('T'):    
                data = hex_to_bytes(in_buf[0:pos])
                sensordata_to_mqtt(data)
            
            in_buf = in_buf[pos+1:]      # and keep the rest
            pos = in_buf.find(LF)  
  
    except MQTTException:
        restart_and_reconnect()
    except OSError as e:
        restart_and_reconnect()
```


Nog verder uitwerken:

* bijhouden van een lijst van lokale nodeID's: alleen voor die nodeIDs moeten de binnenkomende MQTT-berichten doorgestuurd worden.
* ontvangen van MQTT berichten (alleen voor de lokale nodes)
    * omzetten van JSON naar LPP
    * aanroepen van de juiste LPP-coderingsfunctie, op basis van het type-veld. (De identifier die als key gebruikt wordt.)
    * omzetten van LPP naar hex (net als in de microbit iotnode)
    * hex-bytestring doorsturen naar de radio-microbit
    
In de radio-microbit:

* als een uart-bericht binnenkomt:
    * omzetten van hex naar binair
    * versturen van het binaire bericht

NB: de microbit-radio kent alleen een broadcast; de radio-microbit hoeft dit bericht niet te interpreteren.

In [ ]:
hex = '0123456789ABCDEF'

def bytes_to_hex(data):
    res = ""
    for i in data:     
        res = res + hex[i // 16] + hex[i % 16]
    return bytes(res, 'ascii')   

In [ ]:
bytes_to_hex([1,2,3, 255, 254, 253])

(Als algemene aanpak denk ik dat het verstandig is om pas op het laatste moment naar bytes te converteren. De meeste bytes-operaties zijn niet echt handig...)

In [ ]:
bytes_to_hex(b'123')

In [ ]:
bytes_to_hex(bytes([1,2,3, 255, 254, 253]))

In [1]:
hex = '0123456789ABCDEF'

def nibble(char):
    if char >= ord('0') and char <= ord('9'):
        return char - ord('0')
    elif char >= ord('A') and char <= ord('F'):
        return 10 + char - ord('A')
    elif char >= ord('a') and char <= ord('f'):
        return 10 + char - ord('a')
    else:
        return 0

def hex_to_bytes(data):
    # input bytes or string; only hex chars; even length
    # output bytes
    if type(data) == str:
        data = bytes(data, 'ascii')
    res = []
    assert(len(data) % 2 == 0)
    i = 0
    while i+1 < len(data):
        byte = nibble(data[i]) * 16 + nibble(data[i+1])
        i += 2
        res.append(byte)
    return bytes(res)

def bytes_to_hex(data):
    # input: bytes or int list
    # output: hex bytes
    res = ''
    for i in data:
        res += hex[i // 16] + hex[i % 16]
    return bytes(res, 'ascii')    
    

In [2]:
hex_to_bytes(b'010203FFFEFD')

b'\x01\x02\x03\xff\xfe\xfd'

In [3]:
type(b'\x01\x02\x03\xff\xfe\xfd')

bytes

In [4]:
type('ab')

str

In [5]:
bytes_to_hex(b'\x01\x02\x03\xff\xfe\xfd')

b'010203FFFEFD'

In [6]:
xxx = bytes_to_hex(bytes([1,2,3,255]))

In [7]:
xxx[3]

50

In [8]:
xxx

b'010203FF'